In [1]:
pip install gensim

Note: you may need to restart the kernel to use updated packages.


In [2]:
from gensim import corpora, models
import pandas as pd

# データの読み込み

In [3]:
import matplotlib.pyplot as plt

In [4]:
target_data_food = pd.read_csv("conv_cat1_food_data_target_user10_2500.csv", index_col=0)

In [5]:
target_data_food

,receipt_num,receipt_buydate,G_gyoutai_name,cat1,cat4,id
Unnamed: 0,,,,,,
0,53471411.0,2014-05-31,コンビニエンスストア,食料品,たばこ,10020.0
1,53308441.0,2014-05-28,コンビニエンスストア,食料品,たばこ,10020.0
2,53338520.0,2014-05-28,コンビニエンスストア,食料品,たばこ,10020.0
3,53346070.0,2014-05-29,コンビニエンスストア,食料品,たばこ,10020.0
4,53382477.0,2014-05-30,コンビニエンスストア,食料品,たばこ,10020.0
...,...,...,...,...,...,...
1198091,56385167.0,2014-08-07,コンビニエンスストア,食料品,コ－ヒ－ドリンク,24874.0
1198092,56599762.0,2014-08-12,コンビニエンスストア,食料品,コ－ヒ－ドリンク,24874.0
1198100,56186988.0,2014-08-02,コンビニエンスストア,食料品,コ－ヒ－ドリンク,29638.0


# コーパス作成

In [6]:
#全体のコーパス作成
grouped_cat4_data = target_data_food.groupby('id')['cat4'].apply(lambda x: [product.strip() for product in x]).reset_index()
user_cat4 = grouped_cat4_data['cat4'].tolist()

In [7]:
#年月別のコーパス作成
target_data_food['receipt_buydate'] = pd.to_datetime(target_data_food['receipt_buydate'])
grouped_cat4_data_ym = target_data_food.groupby([target_data_food['receipt_buydate'].dt.to_period("M"), 'id'])['cat4'].apply(lambda x: [product.strip() for product in x]).reset_index()
user_cat4_ym = grouped_cat4_data_ym['cat4'].tolist()

In [8]:
grouped_cat4_data_ym

,receipt_buydate,id,cat4
0,2014-04,0.0,"[ビ－ル, スナック, スポ－ツドリンク, スポ－ツドリンク, 果汁飲料]"
1,2014-04,4.0,"[コ－ヒ－ドリンク, コ－ヒ－ドリンク, コ－ヒ－ドリンク, キャンディ, コ－ヒ－ドリンク..."
2,2014-04,8.0,"[炭酸飲料, ｶｯﾌﾟｲﾝｽﾀﾝﾄ麺]"
3,2014-04,11.0,"[スナック, 煎餅・あられ, 炭酸飲料, 乳酸飲料, チョコレ－ト, 炭酸飲料, 炭酸飲料,..."
4,2014-04,16.0,"[たばこ, たばこ, たばこ, たばこ, たばこ, たばこ]"
...,...,...,...
121765,2015-03,40317.0,"[液体茶, アイスクリ－ム]"
121766,2015-03,40326.0,"[チョコレ－ト, 食パン]"
121767,2015-03,40332.0,"[ビ－ル, 冷凍調理, ビ－ル, ビ－ル, 液体茶, コ－ヒ－ドリンク, コ－ヒ－ドリンク,..."
121768,2015-03,40338.0,"[たばこ, たばこ, たばこ, たばこ, たばこ, たばこ, 菓子パン・調理パン, 菓子パン..."


In [9]:
grouped_cat4_data_ym.groupby(grouped_cat4_data_ym['receipt_buydate']).count()

,id,cat4
receipt_buydate,,
2014-04,10207,10207
2014-05,10357,10357
2014-06,10111,10111
2014-07,10287,10287
2014-08,10369,10369
2014-09,10132,10132
2014-10,10250,10250
2014-11,9979,9979
2014-12,9911,9911


In [10]:
# 月ごとに分割して辞書に保存
#grouped_cat4_data_ym['receipt_buydate'] = pd.to_datetime(grouped_cat4_data_ym['receipt_buydate'])
monthly_data = {month: data for month, data in grouped_cat4_data_ym.groupby(grouped_cat4_data_ym['receipt_buydate'])['cat4']}


In [11]:
for month, data in monthly_data.items():
    print(month)

2014-04
2014-05
2014-06
2014-07
2014-08
2014-09
2014-10
2014-11
2014-12
2015-01
2015-02
2015-03


In [12]:
monthly_data[pd.Period('2014-04', freq='M')]

0                    [ビ－ル, スナック, スポ－ツドリンク, スポ－ツドリンク, 果汁飲料]
1        [コ－ヒ－ドリンク, コ－ヒ－ドリンク, コ－ヒ－ドリンク, キャンディ, コ－ヒ－ドリンク...
2                                      [炭酸飲料, ｶｯﾌﾟｲﾝｽﾀﾝﾄ麺]
3        [スナック, 煎餅・あられ, 炭酸飲料, 乳酸飲料, チョコレ－ト, 炭酸飲料, 炭酸飲料,...
4                           [たばこ, たばこ, たばこ, たばこ, たばこ, たばこ]
                               ...                        
10202                                         [デザ－ト類, 食パン]
10203    [液体茶, 食パン, 液体茶, 液体茶, チョコレ－ト, ビ－ル, 美容・健康ドリンク, チ...
10204          [アイスクリ－ム, アイスクリ－ム, アイスクリ－ム, たばこ, 菓子パン・調理パン]
10205    [ﾋﾞｽｹﾂﾄ&ｸﾗﾂｶ-, ﾋﾞｽｹﾂﾄ&ｸﾗﾂｶ-, ﾋﾞｽｹﾂﾄ&ｸﾗﾂｶ-, 炭酸飲...
10206    [畜肉ハム, 冷凍調理, 冷凍調理, ﾋﾞｽｹﾂﾄ&ｸﾗﾂｶ-, スナック, 冷凍農産, 冷...
Name: cat4, Length: 10207, dtype: object

# 辞書の作成

In [13]:
texts = target_data_food['cat4'].tolist()
processed_docs = [[str(doc).lower().strip()] for doc in texts]
dictionary = corpora.Dictionary(processed_docs)

In [14]:
for doc in dictionary.values():
    print(doc)

たばこ
スナック
デザ－ト類
スポ－ツドリンク
チョコレ－ト
アイスクリ－ム
チュ－インガム
コ－ヒ－ドリンク
液体茶
菓子パン・調理パン
その他酒類
かまぼこ
ｶｯﾌﾟｲﾝｽﾀﾝﾄ麺
生麺・ゆで麺
コ－ラ
漬物
栄養ドリンク
キャンディ
ビ－ル
ウィスキ－
つまみ類
その他飲料
ミネラルウォ－タ類
その他スプレッド類
香辛料
シチュ－ベ－ス
サラダ油・天ぷら油
果汁飲料
食パン
チ－ズ
炭酸飲料
カレ－
乳酸菌飲料
野菜ジユ－ス
煎餅・あられ
ﾋﾞｽｹﾂﾄ&ｸﾗﾂｶ-
冷凍調理
焼酎
畜肉ハム
サイダ－
ワイン
袋インスタント麺
まぜご飯の素
冷凍農産
洋風食品
ヨ－グルト
紅茶ドリンク
１００％ジュ－ス
乳酸飲料
ちくわ
ドレッシング
パスタソ－ス
魚貝類缶詰
牛乳
米飯類
ベ－コン
ス－プ類
焼肉ｼｬﾌﾞｼｬﾌﾞのﾀﾚ
和風食品
味噌
味噌汁・吸物類
揚げ物
シリアル類
エッセンス類
美容・健康ドリンク
畜肉ソ－セ－ジ
豆腐類
納豆
シチュ－
食用塩
中華風食品
玩具メ－カ－菓子
シロップ類
食酢
その他調理食品
日本茶
ﾚｷﾞｭﾗ-ｺ-ﾋ-
醤油
その他合わせ調味料
トマトジュ－ス
日本酒
お茶漬けの素
佃煮
乾麺
ふりかけ
畜肉缶詰
もずく・めかぶ
豆乳
栄養バランス食品
魚肉ソ－セ－ジ
海苔
ジャム・ママレ－ド
うまみ調味料
ﾒﾆｭ-専用料理の素
てんぷら粉
キャラメル
つゆ・煮物料理の素
マヨネ－ズ
その他冷凍食品
スパゲッティ－
ゴマ油
冷凍水産
その他菓子
はんぺん
プレミックス
ﾌﾚｯｼｭｸﾘ-ﾑ
その他汎用調味料
麦茶
その他練り製品
小麦粉
フル－ツ缶詰
砂糖
ｲﾝｽﾀﾝﾄｺ-ﾋ-
ｲﾝｽﾀﾝﾄ ｸﾘ-ﾑ
唐揚げ粉
ﾎｲｯﾌﾟｸﾘ-ﾑ
パウチ入り食材
ココア
焼豚
マカロニ類
ソ－ス
パン粉
わかめ・こんぶ類
春雨・くず切り
紅茶
マ－ガリン類
調理用ス－プ
こうや豆腐
野菜缶詰
煮豆
蜂蜜
ケチャップ
削り節
米
バタ－
みりん風調味料
ぽん酢
その他缶詰
その他麺類
練りミルク
低カロリ－甘味料
その他茶
料理酒
介護食品
海藻サラダ
中国茶
本みりん
液体だし
風味調味料
ベビ－フ－ド
煮干し
その他食用油
その他食品
魚肉ハム
鍋補完材
スキムミルク
麦芽飲料
複合調味料


# コーパス関数

In [15]:
#コーパスを作成する関数
def create_corpus(m_data):
    corpus = [dictionary.doc2bow(doc) for doc in m_data]
    return corpus


In [16]:
#月別のデータのコーパス作成

#月別のコーパスを保存する辞書
corpus_dict = {}

for month, data in monthly_data.items():
    print(f'{month}のデータ完了')
    print()
    corpus_dict[month] = data


2014-04のデータ完了

2014-05のデータ完了

2014-06のデータ完了

2014-07のデータ完了

2014-08のデータ完了

2014-09のデータ完了

2014-10のデータ完了

2014-11のデータ完了

2014-12のデータ完了

2015-01のデータ完了

2015-02のデータ完了

2015-03のデータ完了



In [17]:
corpus_dict

{Period('2014-04', 'M'): 0                    [ビ－ル, スナック, スポ－ツドリンク, スポ－ツドリンク, 果汁飲料]
 1        [コ－ヒ－ドリンク, コ－ヒ－ドリンク, コ－ヒ－ドリンク, キャンディ, コ－ヒ－ドリンク...
 2                                      [炭酸飲料, ｶｯﾌﾟｲﾝｽﾀﾝﾄ麺]
 3        [スナック, 煎餅・あられ, 炭酸飲料, 乳酸飲料, チョコレ－ト, 炭酸飲料, 炭酸飲料,...
 4                           [たばこ, たばこ, たばこ, たばこ, たばこ, たばこ]
                                ...                        
 10202                                         [デザ－ト類, 食パン]
 10203    [液体茶, 食パン, 液体茶, 液体茶, チョコレ－ト, ビ－ル, 美容・健康ドリンク, チ...
 10204          [アイスクリ－ム, アイスクリ－ム, アイスクリ－ム, たばこ, 菓子パン・調理パン]
 10205    [ﾋﾞｽｹﾂﾄ&ｸﾗﾂｶ-, ﾋﾞｽｹﾂﾄ&ｸﾗﾂｶ-, ﾋﾞｽｹﾂﾄ&ｸﾗﾂｶ-, 炭酸飲...
 10206    [畜肉ハム, 冷凍調理, 冷凍調理, ﾋﾞｽｹﾂﾄ&ｸﾗﾂｶ-, スナック, 冷凍農産, 冷...
 Name: cat4, Length: 10207, dtype: object,
 Period('2014-05', 'M'): 10207    [液体茶, スナック, その他酒類, スポ－ツドリンク, スポ－ツドリンク, スポ－ツドリン...
 10208    [コ－ヒ－ドリンク, コ－ヒ－ドリンク, コ－ヒ－ドリンク, コ－ヒ－ドリンク, コ－ヒ－ド...
 10209                                                 [牛乳]
 10210                          [コ－ヒ－ドリンク, ミネラルウォ－タ類, 果汁飲料]
 10211   

# ldaの実装

In [18]:
from gensim.models import LdaModel

In [19]:
#ldaモデルの作成
for month, corpus_org in corpus_dict.items():
    corpus = create_corpus(corpus_org)
    lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=10)
    print(month)
    for topic_num , term in lda_model.print_topics():
        top_terms = term.split(" + ")
        top_5_terms = " + ".join(top_terms[0:5])
        print(f"({topic_num}, {top_5_terms})")
    print()

2014-04
(0, 0.316*"キャンディ" + 0.156*"栄養ドリンク" + 0.155*"液体茶" + 0.050*"たばこ" + 0.030*"菓子パン・調理パン")
(1, 0.214*"チョコレ－ト" + 0.134*"ミネラルウォ－タ類" + 0.096*"液体茶" + 0.064*"ﾋﾞｽｹﾂﾄ&ｸﾗﾂｶ-" + 0.064*"ヨ－グルト")
(2, 0.112*"冷凍調理" + 0.109*"煎餅・あられ" + 0.088*"菓子パン・調理パン" + 0.068*"スナック" + 0.061*"ﾋﾞｽｹﾂﾄ&ｸﾗﾂｶ-")
(3, 0.278*"スナック" + 0.188*"ｶｯﾌﾟｲﾝｽﾀﾝﾄ麺" + 0.097*"菓子パン・調理パン" + 0.064*"チョコレ－ト" + 0.044*"アイスクリ－ム")
(4, 0.202*"紅茶ドリンク" + 0.152*"野菜ジユ－ス" + 0.149*"１００％ジュ－ス" + 0.079*"液体茶" + 0.074*"ヨ－グルト")
(5, 0.256*"アイスクリ－ム" + 0.244*"液体茶" + 0.112*"デザ－ト類" + 0.073*"果汁飲料" + 0.031*"チョコレ－ト")
(6, 0.666*"菓子パン・調理パン" + 0.054*"食パン" + 0.038*"コ－ヒ－ドリンク" + 0.028*"液体茶" + 0.022*"ヨ－グルト")
(7, 0.304*"コ－ヒ－ドリンク" + 0.224*"ビ－ル" + 0.114*"その他酒類" + 0.096*"液体茶" + 0.024*"スナック")
(8, 0.716*"たばこ" + 0.080*"コ－ヒ－ドリンク" + 0.029*"菓子パン・調理パン" + 0.025*"アイスクリ－ム" + 0.010*"スナック")
(9, 0.228*"牛乳" + 0.088*"菓子パン・調理パン" + 0.079*"炭酸飲料" + 0.078*"コ－ラ" + 0.058*"その他飲料")

2014-05
(0, 0.558*"コ－ヒ－ドリンク" + 0.100*"コ－ラ" + 0.088*"液体茶" + 0.029*"ミネラルウォ－タ類" + 0.018*"栄養ドリンク")
(1, 0.243*"食パン" + 0.073*"

In [21]:
all_data = []

# 月ごとにLDAモデルからトピックを抽出し、リストに追加
for month, corpus_org in corpus_dict.items():
    corpus = create_corpus(corpus_org)
    lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=10)
    for topic_num, term in lda_model.print_topics():
        top_terms = term.split(" + ")
        formatted_terms = ["{}({})".format(word.split("*")[1].replace('"', '').strip(), round(float(word.split("*")[0]), 3)) for word in top_terms[:5]]
        all_data.append([month, topic_num] + formatted_terms)


# データフレームの作成
columns = ['月', 'トピック', '1位', '2位', '3位', '4位', '5位']
df = pd.DataFrame(all_data, columns=columns)

# # CSVファイルへの書き出し
csv_file_path = 'lda_topic/Topics_Data_conv.csv'
df.to_csv(csv_file_path, index=False)

In [ ]:
# LDAモデルの学習
num_topics = 10
# lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics, passes=20, iterations=400)

# 各ドキュメントのトピック割合を取得
doc_topics = [lda_model.get_document_topics(doc) for doc in corpus]

# 各ドキュメントのトピック割合が最も高いトピックをそのドキュメントのトピックとして割り当て
doc_topic_labels = [max(topics, key=lambda x: x[1])[0] for topics in doc_topics]

# トピック割り当ての集計
topic_counts = pd.Series(doc_topic_labels).value_counts()

# 可視化
fig, ax = plt.subplots(figsize=(10, 6))
topic_counts.plot(kind='bar', ax=ax)
ax.set_xlabel('Topic')
ax.set_ylabel('Number of Documents')
ax.set_title(f'Document Topic Distribution (Total Topics: {num_topics})')

plt.tight_layout()
plt.show()

In [ ]:
lda_model

In [ ]:
for month, corpus_org in corpus_dict.items():
    corpus = create_corpus(corpus_org)
    lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=10)
    
    # コーパス全体でのトピック分布を計算
    doc_topics = []
    for doc in corpus:
        doc_topic = lda_model.get_document_topics(doc, minimum_probability=0)
        doc_topics.append([prob for _, prob in sorted(doc_topic)])
        
    # 平均トピック分布を計算
    avg_topic_dist = np.mean(doc_topics, axis=0)
    topic_distributions[month] = avg_topic_dist
        

In [ ]:
len(corpus)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# トピック分布を格納するための辞書
topic_distributions = {}

# 各月のトピック分布を計算
for month, corpus_org in corpus_dict.items():
    corpus = create_corpus(corpus_org)
    lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=10)
    
    # コーパス全体でのトピック分布を計算
    doc_topics = []
    for doc in corpus:
        doc_topic = lda_model.get_document_topics(doc, minimum_probability=0)
        doc_topics.append([prob for _, prob in sorted(doc_topic)])
    
    # 平均トピック分布を計算
    avg_topic_dist = np.mean(doc_topics, axis=0)
    topic_distributions[month] = avg_topic_dist

# データフレームに変換
df = pd.DataFrame.from_dict(topic_distributions, orient='index')
df.columns = [f'Topic {i+1}' for i in range(10)]

# ヒートマップの作成
plt.figure(figsize=(12, 8))
sns.heatmap(df, cmap='YlOrRd', annot=True, fmt='.3f', cbar_kws={'label': 'Topic Proportion'})
plt.title('Monthly Topic Distribution')
plt.xlabel('Topics')
plt.ylabel('Months')
plt.tight_layout()
plt.show()

# 積み上げ棒グラフの作成
plt.figure(figsize=(12, 6))
df.plot(kind='bar', stacked=True)
plt.title('Monthly Topic Distribution')
plt.xlabel('Months')
plt.ylabel('Topic Proportion')
plt.legend(title='Topics', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

# # 折れ線グラフの作成
# plt.figure(figsize=(12, 6))
# for topic in df.columns:
#     plt.plot(df.index, df[topic], marker='o', label=topic)
# plt.title('Topic Trends Over Time')
# plt.xlabel('Months')
# plt.ylabel('Topic Proportion')
# plt.legend(title='Topics', bbox_to_anchor=(1.05, 1), loc='upper left')
# plt.grid(True)
# plt.tight_layout()
# plt.show()

# トピックの詳細情報を表示
print("\nTop words for each topic:")
for month, corpus_org in corpus_dict.items():
    print(f"\nMonth: {month}")
    corpus = create_corpus(corpus_org)
    lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=10)
    for idx, topic in enumerate(lda_model.show_topics(formatted=False, num_words=5)):
        words = [w[0] for w in topic[1]]
        print(f"Topic {idx + 1}: {', '.join(words)}")

In [ ]:
df

In [ ]:
# 折れ線グラフの作成
plt.figure(figsize=(12, 6))
for topic in df.columns:
    plt.plot(df.index.astype("str"), df[topic], marker='o', label=topic)
plt.title('Topic Trends Over Time')
plt.xlabel('Months')
plt.ylabel('Topic Proportion')
plt.legend(title='Topics', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
df.index.astype("str")
